In [ ]:
import requests
import pandas as pd
API_KEY = "USE-YOUR-KEY"
G_API_KEY = "USE-YOUR-KEY"
BASE_URL = "https://api.rentcast.io/v1/listings/rental/long-term"
headers = {
    "X-Api-Key": API_KEY
}

In [31]:
def get_lat_lng(address, api_key):

    geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(geocode_url)
    
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            lat = data['results'][0]['geometry']['location']['lat']
            lng = data['results'][0]['geometry']['location']['lng']
            return lat, lng
        else:
            print("Error: No results found.")
            return None, None
    else:
        print(f"Error: {response.status_code}")
        return None, None

address = "1600 Amphitheatre Parkway, Mountain View, CA"
latitude, longitude = get_lat_lng(address, G_API_KEY)

print(f"Latitude: {latitude}, Longitude: {longitude}")


Latitude: 37.4225022, Longitude: -122.0847398


In [32]:
def get_rental_listings(params):

    response = requests.get(BASE_URL, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve data. Error: {response.status_code} - {response.text}")


In [33]:
params = {
    "latitude": latitude,
    "longitude": longitude,
    'radius': 2000,
    'limit': 100
}
listings = get_rental_listings(params)
print(listings)


[{'id': '90-Lund-Ave,-Unit-1B,-Hayward,-CA-94544', 'formattedAddress': '90 Lund Ave, Unit 1B, Hayward, CA 94544', 'addressLine1': '90 Lund Ave', 'addressLine2': 'Unit 1B', 'city': 'Hayward', 'state': 'CA', 'zipCode': '94544', 'county': 'Alameda', 'latitude': 37.650475, 'longitude': -122.081062, 'propertyType': 'Apartment', 'bedrooms': 1, 'bathrooms': 1, 'squareFootage': 720, 'status': 'Active', 'price': 1895, 'listingType': 'Standard', 'listedDate': '2024-12-14T00:00:00.000Z', 'removedDate': None, 'createdDate': '2024-08-19T00:00:00.000Z', 'lastSeenDate': '2024-12-14T04:38:12.937Z', 'daysOnMarket': 1, 'history': {'2024-12-14': {'event': 'Rental Listing', 'price': 1895, 'listingType': 'Standard', 'listedDate': '2024-12-14T00:00:00.000Z', 'removedDate': None, 'daysOnMarket': 1}}}, {'id': '8130-Baldwin-St,-Unit-A,-Oakland,-CA-94621', 'formattedAddress': '8130 Baldwin St, Unit A, Oakland, CA 94621', 'addressLine1': '8130 Baldwin St', 'addressLine2': 'Unit A', 'city': 'Oakland', 'state': 'C

In [34]:

df=pd.DataFrame(listings)
df

,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,longitude,...,status,price,listingType,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket,history,yearBuilt
0,"90-Lund-Ave,-Unit-1B,-Hayward,-CA-94544","90 Lund Ave, Unit 1B, Hayward, CA 94544",90 Lund Ave,Unit 1B,Hayward,CA,94544,Alameda,37.650475,-122.081062,...,Active,1895,Standard,2024-12-14T00:00:00.000Z,None,2024-08-19T00:00:00.000Z,2024-12-14T04:38:12.937Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
1,"8130-Baldwin-St,-Unit-A,-Oakland,-CA-94621","8130 Baldwin St, Unit A, Oakland, CA 94621",8130 Baldwin St,Unit A,Oakland,CA,94621,Alameda,37.748005,-122.195114,...,Active,5000,Standard,2024-12-14T00:00:00.000Z,None,2024-06-27T00:00:00.000Z,2024-12-14T04:38:12.936Z,1,"{'2024-09-26': {'event': 'Rental Listing', 'pr...",NaN
2,"1678-A-Dwight-Way,-Berkeley,-CA-94703","1678-A Dwight Way, Berkeley, CA 94703",1678-A Dwight Way,None,Berkeley,CA,94703,Alameda,37.862446,-122.276894,...,Active,2195,Standard,2024-12-14T00:00:00.000Z,None,2024-08-25T00:00:00.000Z,2024-12-14T04:38:12.933Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
3,"620-S-K-St,-Livermore,-CA-94550","620 S K St, Livermore, CA 94550",620 S K St,None,Livermore,CA,94550,Alameda,37.677208,-121.767749,...,Active,3650,Standard,2024-12-14T00:00:00.000Z,None,2019-07-19T10:25:55.152Z,2024-12-14T04:38:12.929Z,1,"{'2024-10-28': {'event': 'Rental Listing', 'pr...",NaN
4,"233-Broadway,-Unit-207,-Oakland,-CA-94607","233 Broadway, Unit 207, Oakland, CA 94607",233 Broadway,Unit 207,Oakland,CA,94607,Alameda,37.796753,-122.277016,...,Active,1627,Standard,2024-12-14T00:00:00.000Z,None,2024-12-11T00:00:00.000Z,2024-12-14T04:38:12.929Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"220-Yampa-Way,-Fremont,-CA-94539","220 Yampa Way, Fremont, CA 94539",220 Yampa Way,None,Fremont,CA,94539,Alameda,37.464017,-121.911550,...,Active,3650,Standard,2024-12-14T00:00:00.000Z,None,2020-11-18T00:15:42.017Z,2024-12-14T04:35:56.432Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
96,"37208-Yolo-Ter,-Fremont,-CA-94536","37208 Yolo Ter, Fremont, CA 94536",37208 Yolo Ter,None,Fremont,CA,94536,Alameda,37.552472,-122.013412,...,Active,2750,Standard,2024-12-14T00:00:00.000Z,None,2024-06-11T00:00:00.000Z,2024-12-14T04:35:56.432Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
97,"15637-Anchorage-Cir,-San-Leandro,-CA-94580","15637 Anchorage Cir, San Leandro, CA 94580",15637 Anchorage Cir,None,San Leandro,CA,94580,Alameda,37.676393,-122.156629,...,Active,3900,Standard,2024-12-14T00:00:00.000Z,None,2024-12-06T00:00:00.000Z,2024-12-14T04:35:56.431Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
98,"33000-Mckeown-St,-Union-City,-CA-94587","33000 Mckeown St, Union City, CA 94587",33000 Mckeown St,None,Union City,CA,94587,Alameda,37.569168,-122.068867,...,Active,4300,Standard,2024-12-14T00:00:00.000Z,None,2024-12-07T00:00:00.000Z,2024-12-14T04:35:56.430Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN


In [36]:
df.columns

Index(['id', 'formattedAddress', 'addressLine1', 'addressLine2', 'city',
       'state', 'zipCode', 'county', 'latitude', 'longitude', 'propertyType',
       'bedrooms', 'bathrooms', 'squareFootage', 'status', 'price',
       'listingType', 'listedDate', 'removedDate', 'createdDate',
       'lastSeenDate', 'daysOnMarket', 'history', 'yearBuilt'],
      dtype='object')

In [38]:
df[df['status']=='Active']

,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,longitude,...,status,price,listingType,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket,history,yearBuilt
0,"90-Lund-Ave,-Unit-1B,-Hayward,-CA-94544","90 Lund Ave, Unit 1B, Hayward, CA 94544",90 Lund Ave,Unit 1B,Hayward,CA,94544,Alameda,37.650475,-122.081062,...,Active,1895,Standard,2024-12-14T00:00:00.000Z,None,2024-08-19T00:00:00.000Z,2024-12-14T04:38:12.937Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
1,"8130-Baldwin-St,-Unit-A,-Oakland,-CA-94621","8130 Baldwin St, Unit A, Oakland, CA 94621",8130 Baldwin St,Unit A,Oakland,CA,94621,Alameda,37.748005,-122.195114,...,Active,5000,Standard,2024-12-14T00:00:00.000Z,None,2024-06-27T00:00:00.000Z,2024-12-14T04:38:12.936Z,1,"{'2024-09-26': {'event': 'Rental Listing', 'pr...",NaN
2,"1678-A-Dwight-Way,-Berkeley,-CA-94703","1678-A Dwight Way, Berkeley, CA 94703",1678-A Dwight Way,None,Berkeley,CA,94703,Alameda,37.862446,-122.276894,...,Active,2195,Standard,2024-12-14T00:00:00.000Z,None,2024-08-25T00:00:00.000Z,2024-12-14T04:38:12.933Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
3,"620-S-K-St,-Livermore,-CA-94550","620 S K St, Livermore, CA 94550",620 S K St,None,Livermore,CA,94550,Alameda,37.677208,-121.767749,...,Active,3650,Standard,2024-12-14T00:00:00.000Z,None,2019-07-19T10:25:55.152Z,2024-12-14T04:38:12.929Z,1,"{'2024-10-28': {'event': 'Rental Listing', 'pr...",NaN
4,"233-Broadway,-Unit-207,-Oakland,-CA-94607","233 Broadway, Unit 207, Oakland, CA 94607",233 Broadway,Unit 207,Oakland,CA,94607,Alameda,37.796753,-122.277016,...,Active,1627,Standard,2024-12-14T00:00:00.000Z,None,2024-12-11T00:00:00.000Z,2024-12-14T04:38:12.929Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"220-Yampa-Way,-Fremont,-CA-94539","220 Yampa Way, Fremont, CA 94539",220 Yampa Way,None,Fremont,CA,94539,Alameda,37.464017,-121.911550,...,Active,3650,Standard,2024-12-14T00:00:00.000Z,None,2020-11-18T00:15:42.017Z,2024-12-14T04:35:56.432Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
96,"37208-Yolo-Ter,-Fremont,-CA-94536","37208 Yolo Ter, Fremont, CA 94536",37208 Yolo Ter,None,Fremont,CA,94536,Alameda,37.552472,-122.013412,...,Active,2750,Standard,2024-12-14T00:00:00.000Z,None,2024-06-11T00:00:00.000Z,2024-12-14T04:35:56.432Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
97,"15637-Anchorage-Cir,-San-Leandro,-CA-94580","15637 Anchorage Cir, San Leandro, CA 94580",15637 Anchorage Cir,None,San Leandro,CA,94580,Alameda,37.676393,-122.156629,...,Active,3900,Standard,2024-12-14T00:00:00.000Z,None,2024-12-06T00:00:00.000Z,2024-12-14T04:35:56.431Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
98,"33000-Mckeown-St,-Union-City,-CA-94587","33000 Mckeown St, Union City, CA 94587",33000 Mckeown St,None,Union City,CA,94587,Alameda,37.569168,-122.068867,...,Active,4300,Standard,2024-12-14T00:00:00.000Z,None,2024-12-07T00:00:00.000Z,2024-12-14T04:35:56.430Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN


In [42]:
df.sort_values(by='price',inplace=True)

In [43]:
df

,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,longitude,...,status,price,listingType,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket,history,yearBuilt
26,"2605-High-St,-Unit-2605,-Oakland,-CA-94601","2605 High St, Unit 2605, Oakland, CA 94601",2605 High St,Unit 2605,Oakland,CA,94601,Alameda,37.782777,-122.204223,...,Active,795,Standard,2024-12-14T00:00:00.000Z,None,2024-11-19T00:00:00.000Z,2024-12-14T04:35:56.562Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
6,"9226-Bancroft-Ave,-Unit-9226,-Oakland,-CA-94603","9226 Bancroft Ave, Unit 9226, Oakland, CA 94603",9226 Bancroft Ave,Unit 9226,Oakland,CA,94603,Alameda,37.752411,-122.164314,...,Active,850,Standard,2024-12-14T00:00:00.000Z,None,2024-11-13T00:00:00.000Z,2024-12-14T04:38:12.926Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
14,"3433-Haven-St,-Unit-3433,-Oakland,-CA-94608","3433 Haven St, Unit 3433, Oakland, CA 94608",3433 Haven St,Unit 3433,Oakland,CA,94608,Alameda,37.826687,-122.283587,...,Active,899,Standard,2024-12-14T00:00:00.000Z,None,2024-11-13T00:00:00.000Z,2024-12-14T04:38:12.913Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
56,"727-20th-St,-Unit-A,-Oakland,-CA-94612","727 20th St, Unit A, Oakland, CA 94612",727 20th St,Unit A,Oakland,CA,94612,Alameda,37.811020,-122.276154,...,Active,1406,Standard,2024-12-14T00:00:00.000Z,None,2024-10-24T00:00:00.000Z,2024-12-14T04:35:56.470Z,1,"{'2024-11-21': {'event': 'Rental Listing', 'pr...",NaN
15,"2554-E-16th-St,-Apt-11,-Oakland,-CA-94601","2554 E 16th St, Apt 11, Oakland, CA 94601",2554 E 16th St,Apt 11,Oakland,CA,94601,Alameda,37.783730,-122.230492,...,Active,1473,Standard,2024-12-14T00:00:00.000Z,None,2024-10-09T00:00:00.000Z,2024-12-14T04:38:12.909Z,1,"{'2024-10-14': {'event': 'Rental Listing', 'pr...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"2800-Kelsey-St,-Berkeley,-CA-94705","2800 Kelsey St, Berkeley, CA 94705",2800 Kelsey St,None,Berkeley,CA,94705,Alameda,37.859865,-122.250359,...,Active,5750,Standard,2024-12-14T00:00:00.000Z,None,2020-12-30T01:37:40.405Z,2024-12-14T04:35:56.433Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
36,"2425-Ellsworth-St,---2425,-Berkeley,-CA-94704","2425 Ellsworth St, # 2425, Berkeley, CA 94704",2425 Ellsworth St,# 2425,Berkeley,CA,94704,Alameda,37.865321,-122.262931,...,Active,6475,Standard,2024-12-14T00:00:00.000Z,None,2024-07-25T00:00:00.000Z,2024-12-14T04:35:56.496Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
82,"406-Singleton-Ave,-Alameda,-CA-94501","406 Singleton Ave, Alameda, CA 94501",406 Singleton Ave,None,Alameda,CA,94501,Alameda,37.787253,-122.282513,...,Active,7000,Standard,2024-12-14T00:00:00.000Z,None,2024-09-05T00:00:00.000Z,2024-12-14T04:35:56.443Z,1,"{'2024-10-01': {'event': 'Rental Listing', 'pr...",NaN
40,"245-Sandringham-Rd,-Piedmont,-CA-94611","245 Sandringham Rd, Piedmont, CA 94611",245 Sandringham Rd,None,Piedmont,CA,94611,Alameda,37.813917,-122.215336,...,Active,8600,Standard,2024-12-14T00:00:00.000Z,None,2022-01-14T02:28:38.535Z,2024-12-14T04:35:56.493Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",1954.0


In [62]:
price_df = df.reset_index()

In [64]:
price_df = price_df.rename(columns={'index': 'price_id'})

In [65]:
df_homes = price_df[['latitude','longitude']]
df_homes

,latitude,longitude
0,37.782777,-122.204223
1,37.752411,-122.164314
2,37.826687,-122.283587
3,37.811020,-122.276154
4,37.783730,-122.230492
...,...,...
95,37.859865,-122.250359
96,37.865321,-122.262931
97,37.787253,-122.282513
98,37.813917,-122.215336


In [53]:
houses = [
    {"id": idx, "location": (row['latitude'], row['longitude'])}
    for idx, row in df_homes.iterrows()
]

In [54]:
houses

[{'id': 26, 'location': (37.782777, -122.204223)},
 {'id': 6, 'location': (37.752411, -122.164314)},
 {'id': 14, 'location': (37.826687, -122.283587)},
 {'id': 56, 'location': (37.81102, -122.276154)},
 {'id': 15, 'location': (37.78373, -122.230492)},
 {'id': 10, 'location': (37.813896, -122.267265)},
 {'id': 17, 'location': (37.796753, -122.277016)},
 {'id': 20, 'location': (37.796753, -122.277016)},
 {'id': 59, 'location': (37.875832, -122.266228)},
 {'id': 7, 'location': (37.796753, -122.277016)},
 {'id': 8, 'location': (37.796753, -122.277016)},
 {'id': 52, 'location': (37.767633, -122.196906)},
 {'id': 4, 'location': (37.796753, -122.277016)},
 {'id': 12, 'location': (37.796753, -122.277016)},
 {'id': 54, 'location': (37.734421, -122.158687)},
 {'id': 65, 'location': (37.813146, -122.242996)},
 {'id': 13, 'location': (37.826111, -122.251816)},
 {'id': 5, 'location': (37.796753, -122.277016)},
 {'id': 47, 'location': (37.837816, -122.303922)},
 {'id': 94, 'location': (37.575729, -1

In [55]:
import pickle as pkl
with open('houses.pkl','wb') as f:
    pkl.dump(houses,f)

## Post evaluation after looking at results from Google-Maps code

In [68]:
#Using average distances, the best house is 23
price_df[price_df['price_id']==23]

,price_id,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,...,status,price,listingType,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket,history,yearBuilt
88,23,"119-Orr-Rd,-Alameda,-CA-94502","119 Orr Rd, Alameda, CA 94502",119 Orr Rd,None,Alameda,CA,94502,Alameda,37.736413,...,Active,4895,Standard,2024-12-14T00:00:00.000Z,None,2023-03-05T00:00:00.000Z,2024-12-14T04:35:56.564Z,1,"{'2024-10-10': {'event': 'Rental Listing', 'pr...",NaN


In [69]:
#Using weighted distances, the best house is 52
price_df[price_df['price_id']==52]

,price_id,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,...,status,price,listingType,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket,history,yearBuilt
11,52,"1759-Seminary-Ave,-Oakland,-CA-94621","1759 Seminary Ave, Oakland, CA 94621",1759 Seminary Ave,None,Oakland,CA,94621,Alameda,37.767633,...,Active,1595,Standard,2024-12-14T00:00:00.000Z,None,2019-11-03T21:27:51.179Z,2024-12-14T04:35:56.475Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",1930.0


In [70]:
#Using k-NN, the best house is 17
price_df[price_df['price_id']==17]

,price_id,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,...,status,price,listingType,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket,history,yearBuilt
6,17,"233-Broadway,-Unit-308,-Oakland,-CA-94607","233 Broadway, Unit 308, Oakland, CA 94607",233 Broadway,Unit 308,Oakland,CA,94607,Alameda,37.796753,...,Active,1541,Standard,2024-12-14T00:00:00.000Z,None,2024-09-30T00:00:00.000Z,2024-12-14T04:38:12.907Z,1,"{'2024-12-14': {'event': 'Rental Listing', 'pr...",NaN
